In [ ]:
# import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_samples, silhouette_score, calinski_harabasz_score

In [ ]:
full_2015_county_data = pd.read_csv("data/full_2015_county_data.csv")
full_2015_county_data

In [ ]:
full_2017_county_data = pd.read_csv("data/full_2017_county_data.csv")

full_2017_county_data

In [ ]:
full_unsupervised_2015 = full_2015_county_data.iloc[:, 6:]
full_unsupervised_2015

In [ ]:
full_unsupervised_2017 = full_2017_county_data.iloc[:, 6:]
full_unsupervised_2017

In [ ]:
full_unsupervised_2015['Men'] = full_unsupervised_2015['Men']/full_unsupervised_2015['TotalPop']
full_unsupervised_2015['Women'] = full_unsupervised_2015['Women']/full_unsupervised_2015['TotalPop']
full_unsupervised_2015['Citizen'] = full_unsupervised_2015['Citizen']/full_unsupervised_2015['TotalPop']

full_unsupervised_2015 = full_unsupervised_2015.fillna(0)

full_unsupervised_2017['Men'] = full_unsupervised_2017['Men']/full_unsupervised_2017['TotalPop']
full_unsupervised_2017['Women'] = full_unsupervised_2017['Women']/full_unsupervised_2017['TotalPop']
full_unsupervised_2017['VotingAgeCitizen'] = full_unsupervised_2017['VotingAgeCitizen']/full_unsupervised_2017['TotalPop']

full_unsupervised_2017 = full_unsupervised_2017.fillna(0)

In [ ]:
# Normalize features

full_normalized_2015_dataset = (full_unsupervised_2015-full_unsupervised_2015.min())/(full_unsupervised_2015.max()-full_unsupervised_2015.min())
full_normalized_2015_dataset

In [ ]:
# full_normalized_2015_dataset.max()

In [ ]:
# full_normalized_2015_dataset.min()

In [ ]:
full_normalized_2017_dataset = (full_unsupervised_2017-full_unsupervised_2017.min())/(full_unsupervised_2017.max()-full_unsupervised_2017.min())
full_normalized_2017_dataset

In [ ]:
# full_normalized_2017_dataset.max()

In [ ]:
# full_normalized_2017_dataset.min()

In [ ]:
full_county_identifier_2015 = full_2015_county_data.iloc[:, :6]
full_county_identifier_2015 

In [ ]:
full_county_identifier_2017 = full_2017_county_data.iloc[:, :6]
full_county_identifier_2017 

In [ ]:
continental_2015_county_data = pd.read_csv("data/continental_2015_county_data.csv")
continental_2015_county_data

In [ ]:
continental_2017_county_data = pd.read_csv("data/continental_2017_county_data.csv")
continental_2017_county_data

In [ ]:
continental_unsupervised_2015 = continental_2015_county_data.iloc[:, 6:]
continental_unsupervised_2015

In [ ]:
continental_unsupervised_2017 = continental_2017_county_data.iloc[:, 6:]
continental_unsupervised_2017

In [ ]:
continental_normalized_2015_dataset = (continental_unsupervised_2015-continental_unsupervised_2015.min())/(continental_unsupervised_2015.max()-continental_unsupervised_2015.min())
continental_normalized_2015_dataset

In [ ]:
# continental_normalized_2015_dataset.max()

In [ ]:
# continental_normalized_2015_dataset.min()

In [ ]:
continental_normalized_2017_dataset = (continental_unsupervised_2017-continental_unsupervised_2017.min())/(continental_unsupervised_2017.max()-continental_unsupervised_2017.min())
continental_normalized_2017_dataset

In [ ]:
# continental_normalized_2017_dataset.max()

In [ ]:
# continental_normalized_2017_dataset.min()

In [ ]:
continental_county_identifier_2015 = continental_2015_county_data.iloc[:, :6]
continental_county_identifier_2015 

In [ ]:
continental_county_identifier_2017 = continental_2017_county_data.iloc[:, :6]
continental_county_identifier_2017 

In [ ]:
full_normalized_2017_dataset

In [ ]:
# full_normalized_2017_dataset.isnull().sum()

In [ ]:
for i in range(len(full_normalized_2017_dataset)):
    if full_normalized_2017_dataset['ChildPoverty'].isnull()[i] == True:
        print(i)
full_normalized_2017_dataset = full_normalized_2017_dataset.drop(labels=[548], axis=0)

# Ridge and Lasso Regression

In [ ]:
from sklearn import linear_model

In [ ]:
# Ridge
n_alphas = 200
alpha_vals = np.logspace(-5, 3, n_alphas)

df_top_feats = pd.DataFrame()

coefs_r = {}
max_feats_r = {}
for c in range(len(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns)):
    coefs_r[c] = []
    max_feats_r[c] = []
    for alpha in alpha_vals:
        ridge_model = linear_model.Ridge(alpha=alpha)
        ridge_model.fit(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].values, full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]][full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns[c]].values)
        coefs_r[c].append(ridge_model.coef_)

    max_list = sorted(((value, index) for index, value in enumerate(coefs_r[c][0])), reverse=True)
    for i in range(10):
        max_feats_r[c].append(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns[max_list[i][1]])
    df_top_feats[full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns[c]] = max_feats_r[c]
        
        
  # TODO: 
  # - label top features for each plot
  # normalize weights
  # plot as fn of ||beta_alpha^R||_2 / ||beta||_2

In [ ]:
df_top_feats

In [ ]:
# Check how many times each feature appears in top 5 
# normalize - each feature is likely a top feature for its own predictor
feats = []
for c in range(len(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns)):
    for i in range(df_top_feats.shape[0]):
        feats.append(df_top_feats[[full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].columns[c]]].iloc[i][0])

In [ ]:
pd.Series(feats).value_counts().plot('bar')

In [ ]:
for c in range(len(full_normalized_2017_dataset.columns)):
    ax = plt.gca()

    ax.plot(alpha_vals, coefs_r[c])
    ax.set_xscale("log")
    ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    plt.xlabel("alpha")
    plt.ylabel("weights")
    plt.title(f"Ridge coefficients as a function of the regularization ({full_normalized_2017_dataset.columns[c]})")
    plt.axis("tight")
    plt.show()
    
    print('Top Features:')
    print(max_feats_r[c])

In [ ]:
# Lasso
n_alphas = 200
alpha_vals = np.logspace(-5, 3, n_alphas)

coefs_l = {}
max_feats_l = {}
for c in range(len(full_normalized_2017_dataset.columns)):
    coefs_l[c] = []
    max_feats_l[c] = []
    for alpha in alpha_vals:
        lasso_model = linear_model.Lasso(alpha=alpha)
        lasso_model.fit(full_normalized_2017_dataset.values, full_normalized_2017_dataset[full_normalized_2017_dataset.columns[c]].values)
        coefs_l[c].append(lasso_model.coef_)

    max_list = sorted(((value, index) for index, value in enumerate(coefs_l[c][0])), reverse=True)
    for i in range(5):
        max_feats_l[c].append(full_normalized_2017_dataset.columns[max_list[i][1]])
        
        
  # TODO: 
  # - label top features for each plot
  # normalize weights
  # plot as fn of ||beta_alpha^R||_2 / ||beta||_2

In [ ]:
for c in range(len(full_normalized_2017_dataset.columns)):
    ax = plt.gca()

    ax.plot(alpha_vals, coefs_l[c])
    ax.set_xscale("log")
    ax.set_xlim(ax.get_xlim()[::-1])  # reverse axis
    plt.xlabel("alpha")
    plt.ylabel("weights")
    plt.title(f"Lasso coefficients as a function of the regularization ({full_normalized_2017_dataset.columns[c]})")
    plt.axis("tight")
    plt.show()
    
    print('Top Features:')
    print(max_feats_l[c])

In [ ]:
# PCA
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA, KernelPCA
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
def apply_pca(image_df, seed):
#     np.random.seed(seed)
#     rndperm = np.random.permutation(image_df.shape[0])
#     df_subset = image_df.loc[rndperm[:sample_size],:].copy()
    df_subset = image_df.copy()
    
    n_components = df_subset.shape[1]
    pca1 = PCA(n_components=n_components)
    pca1.fit(df_subset.values)

    # Scree plot / Pareto plot
    f = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(3, 2)

    ax1 = plt.subplot(gs[0,0])
    ax1.scatter([i for i in range(1,n_components+1)], pca1.singular_values_)

    ax2 = plt.subplot(gs[0,1])
    ax2.bar([i for i in range(1,n_components+1)], pca1.explained_variance_ratio_ *100)
    ax2.plot([i for i in range(1,n_components+1)], np.cumsum(pca1.explained_variance_ratio_ *100), color='red')

    ax1.set_title('Scree Plot')
    ax2.set_title('Pareto Plot')

    ax1.set_ylabel('Eigenvalue')
    ax2.set_ylabel('Variance Explained')

    ax1.set_xlabel('Principal Component')
    ax2.set_xlabel('Principal Component')

    n_components = 3
    pca2 = PCA(n_components=n_components)
    x_new = pca2.fit_transform(df_subset.values)

    ax3 = plt.subplot(gs[1:3,0:2], projection='3d')
    ax3.scatter([i[1] for i in x_new], [i[2] for i in x_new], [i[0] for i in x_new])

    ax3.set_title('PCA (D=3)')
    ax3.set_xlabel('Principal Component 2')
    ax3.set_ylabel('Principal Component 3')
    ax3.set_zlabel('Principal Component 1')
    
#     print('Cumulative Variance: ', np.cumsum(pca2.explained_variance_ratio_ *100))

    plt.tight_layout()
    plt.show()
    
    return pca1

In [ ]:
# PCA
from sklearn.decomposition import PCA
from sklearn.decomposition import PCA, KernelPCA
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
def apply_pca2(image_df, seed):
#     np.random.seed(seed)
#     rndperm = np.random.permutation(image_df.shape[0])
#     df_subset = image_df.loc[rndperm[:sample_size],:].copy()
    df_subset = image_df.copy()
    
    f = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(1, 1)

    n_components = 3
    pca2 = KernelPCA(kernel="sigmoid", fit_inverse_transform=True, n_components=n_components)
    x_new = pca2.fit_transform(df_subset.values)

    ax3 = plt.subplot(gs[0,0], projection='3d')
    ax3.scatter([i[1] for i in x_new], [i[2] for i in x_new], [i[0] for i in x_new])

    ax3.set_title('PCA (D=3)')
    ax3.set_xlabel('Principal Component 2')
    ax3.set_ylabel('Principal Component 3')
    ax3.set_zlabel('Principal Component 1')

    plt.tight_layout()
    plt.show()

In [ ]:
pca1 = apply_pca(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0)

In [ ]:
apply_pca2(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0)

In [ ]:
for i in range(5):
    f = plt.figure(figsize=(20, 12))
    gs = gridspec.GridSpec(2, 2)

    ax1 = plt.subplot(gs[0,0])
    ax1.bar(full_normalized_2017_dataset.columns[:-2], np.abs(pca1.components_[i]))
    ax1.set_title(f'PC{i}')
    plt.xticks(rotation=75)
    plt.show()

In [ ]:
df_pca_var = pd.DataFrame(columns = full_normalized_2017_dataset.columns[:-2])
df_pca_var = df_pca_var.T

for i in range(len(pca1.components_)):
    df_pca_var[i] = pca1.components_[i]

In [ ]:
df_pca_var

# Clustering

In [ ]:
# K-Means
# K-Means
def apply_kmeans(image_df, seed, x, y):
    df_subset = image_df.copy()
    
    image_kmeans = []

    for k in [2,3,4,5,6,7,8,9]:
        image_kmeans.append(KMeans(n_clusters=k, random_state=0).fit(df_subset.values))
        
    colors = {}
    colors[0] = image_kmeans[0].predict(df_subset.values)
    colors[1] = image_kmeans[1].predict(df_subset.values)
    colors[2] = image_kmeans[2].predict(df_subset.values)
    colors[3] = image_kmeans[3].predict(df_subset.values)
    colors[4] = image_kmeans[4].predict(df_subset.values)
    colors[5] = image_kmeans[5].predict(df_subset.values)
    colors[6] = image_kmeans[6].predict(df_subset.values)
    colors[7] = image_kmeans[7].predict(df_subset.values)

    f = plt.figure(figsize=(15, 30))
    gs = gridspec.GridSpec(4, 2)

    ax1 = plt.subplot(gs[0,0])
    ax1.scatter(x, y, c=colors[0], s=2)

    ax2 = plt.subplot(gs[0,1])
    ax2.scatter(x, y, c=colors[1], s=2)

    ax3 = plt.subplot(gs[1,0])
    ax3.scatter(x, y, c=colors[2], s=2)

    ax4 = plt.subplot(gs[1,1])
    ax4.scatter(x, y, c=colors[3], s=2)
    
    ax5 = plt.subplot(gs[2,0])
    ax5.scatter(x, y, c=colors[4], s=2)

    ax6 = plt.subplot(gs[2,1])
    ax6.scatter(x, y, c=colors[5], s=2)
    
    ax7 = plt.subplot(gs[3,0])
    ax7.scatter(x, y, c=colors[6], s=2)

    ax8 = plt.subplot(gs[3,1])
    ax8.scatter(x, y, c=colors[7], s=2)

    ax1.set_title('k=2')
    ax2.set_title('k=3')
    ax3.set_title('k=4')
    ax4.set_title('k=5')
    ax5.set_title('k=6')
    ax6.set_title('k=7')
    ax7.set_title('k=8')
    ax8.set_title('k=9')

    plt.tight_layout()
    plt.show()
    
    k_vals = [2,3,4,5,6,7,8,9]
    silhouette_values = []
    for i,k in enumerate(k_vals):
        sil = silhouette_score(df_subset.values, image_kmeans[i].predict(df_subset.values))
        ch = calinski_harabasz_score(df_subset.values, image_kmeans[i].predict(df_subset.values))

        print(f'k={k}: Silhouette score = {sil}, CH index = {ch}')
        
    return colors

# DBSCAN
# Spectral Clustering

In [ ]:
# HC
def apply_hc(image_df, seed, linkage,x,y):
    df_subset = image_df.copy()
    
    image_hc = []

    for k in [2,3,4,5,6,7,8,9]:
        image_hc.append(AgglomerativeClustering(n_clusters=k, linkage=linkage, affinity='euclidean').fit(df_subset.values))
        
    colors = {}
    colors[0] = image_hc[0].labels_
    colors[1] = image_hc[1].labels_
    colors[2] = image_hc[2].labels_
    colors[3] = image_hc[3].labels_
    colors[4] = image_hc[4].labels_
    colors[5] = image_hc[5].labels_
    colors[6] = image_hc[6].labels_
    colors[7] = image_hc[7].labels_
    
    models = {}
    models[0] = AgglomerativeClustering(n_clusters=2, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[1] = AgglomerativeClustering(n_clusters=3, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[2] = AgglomerativeClustering(n_clusters=4, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[3] = AgglomerativeClustering(n_clusters=5, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[4] = AgglomerativeClustering(n_clusters=6, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[5] = AgglomerativeClustering(n_clusters=7, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[6] = AgglomerativeClustering(n_clusters=8, linkage=linkage, affinity='euclidean').get_params(deep=True)
    models[7] = AgglomerativeClustering(n_clusters=9, linkage=linkage, affinity='euclidean').get_params(deep=True)

    f = plt.figure(figsize=(15, 20))
    gs = gridspec.GridSpec(4, 2)

    ax1 = plt.subplot(gs[0,0])
    ax1.scatter(x, y, c=colors[0], s=2)

    ax2 = plt.subplot(gs[0,1])
    ax2.scatter(x, y, c=colors[1], s=2)

    ax3 = plt.subplot(gs[1,0])
    ax3.scatter(x, y, c=colors[2], s=2)

    ax4 = plt.subplot(gs[1,1])
    ax4.scatter(x, y, c=colors[3], s=2)
    
    ax5 = plt.subplot(gs[2,0])
    ax5.scatter(x, y, c=colors[4], s=2)

    ax6 = plt.subplot(gs[2,1])
    ax6.scatter(x, y, c=colors[5], s=2)
    
    ax7 = plt.subplot(gs[3,0])
    ax7.scatter(x, y, c=colors[6], s=2)

    ax8 = plt.subplot(gs[3,1])
    ax8.scatter(x, y, c=colors[7], s=2)

    ax1.set_title('k=2')
    ax2.set_title('k=3')
    ax3.set_title('k=4')
    ax4.set_title('k=5')
    ax5.set_title('k=6')
    ax6.set_title('k=7')
    ax7.set_title('k=8')
    ax8.set_title('k=9')

    plt.tight_layout()
    plt.show()
    
    k_vals = [2,3,4,5,6,7,8,9]
    silhouette_values = []
    for i,k in enumerate(k_vals):
        sil = silhouette_score(df_subset.values, image_hc[i].labels_)
        ch = calinski_harabasz_score(df_subset.values, image_hc[i].labels_)

        print(f'k={k}: Silhouette score = {sil}, CH index = {ch}')
        
    return models, colors

In [ ]:
# GMM
def apply_gmm(image_df, seed, x, y):
    df_subset = image_df.copy()
    
    image_gmm = []

    for k in [2,3,4,5,6,7]:
        image_gmm.append(GaussianMixture(n_components=k, random_state=0).fit(df_subset.values))
        
    colors = {}
    colors[0] = image_gmm[0].predict(df_subset.values)
    colors[1] = image_gmm[1].predict(df_subset.values)
    colors[2] = image_gmm[2].predict(df_subset.values)
    colors[3] = image_gmm[3].predict(df_subset.values)
    colors[4] = image_gmm[4].predict(df_subset.values)
    colors[5] = image_gmm[5].predict(df_subset.values)

    f = plt.figure(figsize=(15, 20))
    gs = gridspec.GridSpec(3, 2)

    ax1 = plt.subplot(gs[0,0])
    ax1.scatter(x, y, c=colors[0], s=2)

    ax2 = plt.subplot(gs[0,1])
    ax2.scatter(x, y, c=colors[1], s=2)

    ax3 = plt.subplot(gs[1,0])
    ax3.scatter(x, y, c=colors[2], s=2)

    ax4 = plt.subplot(gs[1,1])
    ax4.scatter(x, y, c=colors[3], s=2)
    
    ax5 = plt.subplot(gs[2,0])
    ax5.scatter(x, y, c=colors[4], s=2)

    ax6 = plt.subplot(gs[2,1])
    ax6.scatter(x, y, c=colors[5], s=2)

    ax1.set_title('k=2')
    ax2.set_title('k=3')
    ax3.set_title('k=4')
    ax4.set_title('k=5')
    ax5.set_title('k=6')
    ax6.set_title('k=7')
    
    plt.tight_layout()
    plt.show()
    
    k_vals = [2,3,4,5,6,7]
    silhouette_values = []
    for i,k in enumerate(k_vals):
        sil = silhouette_score(df_subset.values, image_gmm[i].predict(df_subset.values))
        ch = calinski_harabasz_score(df_subset.values, image_gmm[i].predict(df_subset.values))

        print(f'k={k}: Silhouette score = {sil}, CH index = {ch}')
        
    return colors

In [ ]:
def scatter_pca_tsne(image_df, seed, colors, perp_values):
    df_subset = image_df.copy()
    
    f = plt.figure(figsize=(16, 12))
    gs = gridspec.GridSpec(2, 6)
    
    n_components = 3
    pca = PCA(n_components=n_components)
    x_new = pca.fit_transform(df_subset.values)

    ax1 = plt.subplot(gs[0,1:4], projection='3d')
    ax1.scatter([i[1] for i in x_new], [i[2] for i in x_new], [i[0] for i in x_new], c=colors, s=4)

    ax1.set_title('PCA (D=3)')
    ax1.set_xlabel('Principal Component 2')
    ax1.set_ylabel('Principal Component 3')
    ax1.set_zlabel('Principal Component 1')
    
    X_tsne1 = TSNE(n_components=2, perplexity=perp_values[0], learning_rate='auto', init='pca').fit_transform(df_subset.values)
    X_tsne2 = TSNE(n_components=2, perplexity=perp_values[1], learning_rate='auto', init='pca').fit_transform(df_subset.values)
    X_tsne3 = TSNE(n_components=2, perplexity=perp_values[2], learning_rate='auto', init='pca').fit_transform(df_subset.values)
    
    ax2 = plt.subplot(gs[1,0:2])
    ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=colors, s=4)
    
    ax3 = plt.subplot(gs[1,2:4])
    ax3.scatter([i[0] for i in X_tsne2], [i[1] for i in X_tsne2], c=colors, s=4)
    
    ax4 = plt.subplot(gs[1,4:6])
    ax4.scatter([i[0] for i in X_tsne3], [i[1] for i in X_tsne3], c=colors, s=4)
    
    ax2.set_title(f'TSNE (perp={perp_values[0]})')
    ax3.set_title(f'TSNE (perp={perp_values[1]})')
    ax4.set_title(f'TSNE (perp={perp_values[2]})')
    
    plt.tight_layout()
    plt.show()

In [ ]:
colors_kmeans = {}
colors_kmeans[0] = apply_kmeans(full_normalized_2017_dataset, seed=0, x=df_kmeans['Long'], y=df_kmeans['Lat'])
# k=4 seems to be a reasonable estimate

In [ ]:
scatter_pca_tsne(full_normalized_2017_dataset, seed=0, colors=colors_kmeans[0][5], perp_values = [5, 30, 50])

In [ ]:
# Calculate what each cluster is
df_kmeans = full_normalized_2017_dataset.copy()
df_kmeans['cluster'] = colors_kmeans[0][2]

df_kmeans

In [ ]:
f = plt.figure(figsize=(20, 12))
gs = gridspec.GridSpec(2, 2)

ax1 = plt.subplot(gs[0,0])
ax1.scatter(df_kmeans['Long'], df_kmeans['Lat'], c=df_kmeans['cluster'], s=1.5)

In [ ]:
# Try again with unlabeled data
colors_kmeans = {}
colors_kmeans[0] = apply_kmeans(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0, x=df_kmeans['Long'], y=df_kmeans['Lat'])

In [ ]:
scatter_pca_tsne(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0, colors=colors_kmeans[0][5], perp_values = [5, 30, 50])

In [ ]:
# Calculate what each cluster is
df_kmeans = full_normalized_2017_dataset.copy()
df_kmeans['cluster'] = colors_kmeans[0][5]

df_kmeans

In [ ]:
X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].values)

colors = ['blue', 'green', 'red', 'yellow', 'orange', 'purple', 'cyan']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
sc = ax1.scatter(df_kmeans['Long'], df_kmeans['Lat'], c=[colors[i] for i in df_kmeans['cluster']], s=4)
ax1.legend(handles=sc.legend_elements()[0], labels=[0,1,2,3,4,5])

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_kmeans['cluster']], s=4)

ax1.set_title('K-Means Clustering')
ax2.set_title('K-Means Clustering (t-SNE Visualization)')

In [ ]:
df_kmeans['cluster'].unique()

In [ ]:
means = {}

for c in list(set(df_kmeans['cluster'].unique())):
    df_temp = df_kmeans[df_kmeans['cluster']==c].copy()
    means[c] = df_temp.mean().values
    
df_means = pd.DataFrame()
df_means['cl0'] = means[0]
df_means['cl1'] = means[1]
df_means['cl2'] = means[2]
df_means['cl3'] = means[3]
df_means['cl4'] = means[4]
df_means['cl5'] = means[5]

df_means = df_means.T
df_means.columns = df_temp.columns
df_means = df_means.T

In [ ]:
full_normalized_2017_dataset

In [ ]:
colors_hc = {}
model_hc = {}
model_hc[0], colors_hc[0] = apply_hc(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0, linkage='ward', x=full_normalized_2017_dataset['Long'], y=full_normalized_2017_dataset['Lat'])
model_hc[1], colors_hc[1] = apply_hc(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]], seed=0, linkage='ward', x=full_normalized_2017_dataset_no_pr['Long'], y=full_normalized_2017_dataset_no_pr['Lat'])

In [ ]:
import scipy.cluster.hierarchy as shc
Dendrogram = shc.dendrogram((shc.linkage(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], method ='ward')))

In [ ]:
Dendrogram = shc.dendrogram((shc.linkage(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]], method ='ward')))

In [ ]:
scatter_pca_tsne(full_normalized_2017_dataset, seed=0, colors=colors_hc[0][6], perp_values = [5, 30, 50])

In [ ]:
import matplotlib.colors as mcolors

df_hc = full_normalized_2017_dataset.copy()
df_hc['cluster'] = colors_hc[0][5]

X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]].values)

# colors = ['blue', 'cyan', 'purple', 'yellow', 'red', 'green']
colors = ['blue', 'red', 'purple', 'yellow', 'cyan', 'green', 'orange']
# colors = ['red', 'blue', 'purple', 'yellow', 'cyan', 'green', 'orange', 'brown', 'gray']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
# sc = ax1.scatter(df_hc['Long'], df_hc['Lat'], c=[colors[i] for i in df_hc['cluster']], label=df_hc['cluster'], s=3)
sc = ax1.scatter(df_hc['Long'], df_hc['Lat'], c=df_hc['cluster'], cmap=mcolors.ListedColormap(colors), label=df_hc['cluster'], s=2)
ax1.legend(*sc.legend_elements(num=7))
# ax1.legend(handles=colors, labels=[0,1,2,3,4,5,6])
# legend1 = ax1.legend(*sc.legend_elements(num=7),
#                     loc="upper right", title="Clusters")
# ax1.add_artist(legend1)

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_hc['cluster']], s=4)

ax1.set_title('Hierchical Clustering')
ax2.set_title('Hierchical Clustering (t-SNE Visualization)')

plt.show()

In [ ]:
# Describe important features in clusters
for i in range(7):
    df_subset = df_hc[df_hc['cluster'] == i]
    means = []
    signs = []
    for col in df_subset.columns:
        if col == 'cluster':
            means.append(-1)
            signs.append(-2)
        else:
            means.append(np.abs((df_subset[col].mean() - df_hc[col].mean())))
            signs.append(np.sign((df_subset[col].mean() - df_hc[col].mean())))
   
    df_temp = pd.DataFrame(means).T
    df_temp.columns = df_subset.columns
    df_subset = df_subset.append(df_temp)
    
    df_temp = pd.DataFrame(signs).T
    df_temp.columns = df_subset.columns
    df_subset = df_subset.append(df_temp)
    print(f'Cluster {i}:')
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    print('\n')

In [ ]:
# cn = df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()
# df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()]

In [ ]:
# df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T

In [ ]:
df_hc = full_normalized_2017_dataset_no_pr.copy()
df_hc['cluster'] = colors_hc[1][7]

X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]].values)

# colors = ['blue', 'cyan', 'purple', 'yellow', 'red', 'green']
# colors = ['blue', 'purple', 'red', 'yellow', 'cyan', 'green']
colors = ['red', 'blue', 'purple', 'yellow', 'cyan', 'brown', 'orange', 'gray', 'green']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
sc = ax1.scatter(df_hc['Long'], df_hc['Lat'], c=[colors[i] for i in df_hc['cluster']], s=6)
sc = ax1.scatter(df_hc['Long'], df_hc['Lat'], c=df_hc['cluster'], cmap=mcolors.ListedColormap(colors), label=df_hc['cluster'], s=2)
ax1.legend(*sc.legend_elements(num=9))

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_hc['cluster']], s=4)

ax1.set_title('Hierchical Clustering')
ax2.set_title('Hierarchical Clustering (t-SNE Visualization)')

In [ ]:
# Describe important features in clusters
for i in range(len(df_hc['cluster'].unique())):
    df_subset = df_hc[df_hc['cluster'] == i]
    means = []
    signs = []
    for col in df_subset.columns:
        if col == 'cluster':
            means.append(-1)
            signs.append(-2)
        else:
            means.append(np.abs((df_subset[col].mean() - df_hc[col].mean())))
            signs.append(np.sign((df_subset[col].mean() - df_hc[col].mean())))
   
    df_temp = pd.DataFrame(means).T
    df_temp.columns = df_subset.columns
    df_subset = df_subset.append(df_temp)
    
    df_temp = pd.DataFrame(signs).T
    df_temp.columns = df_subset.columns
    df_subset = df_subset.append(df_temp)
    print(f'Cluster {i}:')
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    df_subset = df_subset.drop(columns=[df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()])
    print(df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax(), df_subset[df_subset['cluster'] == -2][df_subset[df_subset['cluster'] == -1].drop(columns=df_subset.columns[-3:]).iloc[0].T.idxmax()][0])
    print('\n')

In [ ]:
df_hc

In [ ]:
full_2017_county_data_no_pr['cluster'] = df_hc['cluster']

In [ ]:
full_2017_county_data_no_pr[full_2017_county_data_no_pr['cluster']==4].sort_values(by=['TotalPop'])[:20]

In [ ]:
full_2017_county_data_no_pr

In [ ]:
colors_gmm = {}
colors_gmm[0] = apply_gmm(full_normalized_2017_dataset[full_normalized_2017_dataset.columns[:-2]], seed=0, x=full_normalized_2017_dataset['Long'], y=full_normalized_2017_dataset['Lat'])

In [ ]:
scatter_pca_tsne(full_normalized_2017_dataset, seed=0, colors=colors_gmm[0][3], perp_values = [5, 30, 50])

In [ ]:
full_2017_county_data[full_2017_county_data['State Code'] != 'PR']

In [ ]:
full_2017_county_data_no_pr = full_2017_county_data.copy()

full_2017_county_data_no_pr = full_2017_county_data_no_pr[full_2017_county_data_no_pr['State Code'] != 'PR']
full_2017_county_data_no_pr = full_2017_county_data_no_pr[full_2017_county_data_no_pr['State Code'] != 'HI']
full_2017_county_data_no_pr = full_2017_county_data_no_pr[full_2017_county_data_no_pr['State Code'] != 'AK']

full_unsupervised_2017_no_pr = full_2017_county_data_no_pr.iloc[:, 6:]

full_unsupervised_2017_no_pr['Men'] = full_unsupervised_2017_no_pr['Men']/full_unsupervised_2017_no_pr['TotalPop']
full_unsupervised_2017_no_pr['Women'] = full_unsupervised_2017_no_pr['Women']/full_unsupervised_2017_no_pr['TotalPop']
full_unsupervised_2017_no_pr['VotingAgeCitizen'] = full_unsupervised_2017_no_pr['VotingAgeCitizen']/full_unsupervised_2017_no_pr['TotalPop']

full_normalized_2017_dataset_no_pr = (full_unsupervised_2017_no_pr-full_unsupervised_2017_no_pr.min())/(full_unsupervised_2017_no_pr.max()-full_unsupervised_2017_no_pr.min())

full_normalized_2017_dataset_no_pr = full_normalized_2017_dataset_no_pr.fillna(0)

In [ ]:
full_normalized_2017_dataset_no_pr

In [ ]:
# NO PR
colors_kmeans = {}
colors_kmeans[0] = apply_kmeans(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]], seed=0, x=full_normalized_2017_dataset_no_pr['Long'], y=full_normalized_2017_dataset_no_pr['Lat'])



In [ ]:
scatter_pca_tsne(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]], seed=0, colors=colors_kmeans[0][4], perp_values = [5, 30, 50])

In [ ]:
df_kmeans = full_normalized_2017_dataset_no_pr.copy()
df_kmeans['cluster'] = colors_kmeans[0][6]

df_kmeans

In [ ]:
X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]].values)

colors = ['blue', 'red', 'purple', 'yellow', 'orange', 'green']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
sc = ax1.scatter(df_kmeans['Long'], df_kmeans['Lat'], c=[colors[i] for i in df_kmeans['cluster']], s=4)
ax1.legend(handles=sc.legend_elements()[0], labels=[0,1,2,3,4,5])

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_kmeans['cluster']], s=4)

ax1.set_title('K-Means Clustering')
ax2.set_title('K-Means Clustering (t-SNE Visualization)')

In [ ]:
X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]].values)

colors = ['blue', 'red', 'purple', 'yellow', 'cyan', 'green', 'orange', 'brown', 'black']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
sc = ax1.scatter(df_kmeans['Long'], df_kmeans['Lat'], c=[colors[i] for i in df_kmeans['cluster']], s=5)
ax1.legend(handles=sc.legend_elements()[0], labels=[0,1,2,3,4,5])

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_kmeans['cluster']], s=4)

ax1.set_title('K-Means Clustering')
ax2.set_title('K-Means Clustering (t-SNE Visualization)')

In [ ]:
X_tsne1 = TSNE(n_components=2, perplexity=30, learning_rate='auto', init='pca').fit_transform(full_normalized_2017_dataset_no_pr[full_normalized_2017_dataset_no_pr.columns[:-2]].values)

colors = ['blue', 'red', 'purple', 'yellow', 'cyan', 'green', 'orange', 'brown']

f = plt.figure(figsize=(20, 8))
gs = gridspec.GridSpec(1,3)

ax1 = plt.subplot(gs[0,0:2])
sc = ax1.scatter(df_kmeans['Long'], df_kmeans['Lat'], c=[colors[i] for i in df_kmeans['cluster']], s=6)
ax1.legend(handles=sc.legend_elements()[0], labels=[0,1,2,3,4,5])

ax1.set_xlabel('Normalized Long')
ax1.set_ylabel('Normalized Lat')

ax2 = plt.subplot(gs[0,2])
ax2.scatter([i[0] for i in X_tsne1], [i[1] for i in X_tsne1], c=[colors[i] for i in df_kmeans['cluster']], s=4)

ax1.set_title('K-Means Clustering')
ax2.set_title('K-Means Clustering (t-SNE Visualization)')